In [7]:
import requests
from bs4 import BeautifulSoup

url = "https://www.ebay.co.uk/itm/355070893965?epid=238941601&hash=item52abdfdf8d%3Ag%3AgcMAAOSws7tlGBPb&amdata=enc%3AAQAIAAAA0E9Wgox51dUQd2LJI4h1P0PL64D6WHetOmdULudxlyRD4OVylnQOBLMGo51%2Fa2mzpzZMLZzzoG0mvnZkctheOseqxdI4UMfRHtVj%2Br2Qk0J0FbGNTEf%2FZYmIks2amBIrMEhGDjpiYsYpo5sypuUHrfy0WO%2Fc2Bsp3jR3xffxCzr8kdh23ByMvO8vCYk%2FpJzq%2BkJgQKgrDSL%2FriT4tENyhmsM3oxaagcbEZrpnEn85%2Fh3XMl1HL%2B9puUiokB3OnDAshLxF4kPcVty9fxoq7o%2FrTA%3D%7Ctkp%3ABFBMiL6h5O1i&LH_ItemCondition=1000%7C1500"

# Send a GET request to the URL
response = requests.get(url)

# Parse the HTML content
soup = BeautifulSoup(response.content, 'html.parser')

# Find the element with the specified class
product_title_element = soup.find('span', {'class': 'ux-textspans'})

# Get the text from the element
product_title = product_title_element.get_text(strip=True)

print(product_title)


In [22]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Define the URLs
urls = ["https://www.ebay.co.uk/itm/355070893965?epid=238941601&hash=item52abdfdf8d%3Ag%3AgcMAAOSws7tlGBPb&amdata=enc%3AAQAIAAAA0E9Wgox51dUQd2LJI4h1P0PL64D6WHetOmdULudxlyRD4OVylnQOBLMGo51%2Fa2mzpzZMLZzzoG0mvnZkctheOseqxdI4UMfRHtVj%2Br2Qk0J0FbGNTEf%2FZYmIks2amBIrMEhGDjpiYsYpo5sypuUHrfy0WO%2Fc2Bsp3jR3xffxCzr8kdh23ByMvO8vCYk%2FpJzq%2BkJgQKgrDSL%2FriT4tENyhmsM3oxaagcbEZrpnEn85%2Fh3XMl1HL%2B9puUiokB3OnDAshLxF4kPcVty9fxoq7o%2FrTA%3D%7Ctkp%3ABFBMiL6h5O1i&LH_ItemCondition=1000%7C1500"]

# Initialize an empty list to store the results
results = []

for url in urls:
    # Send a GET request to the URL
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find the elements
    product_title_element = soup.find('span', {'class': 'ux-textspans ux-textspans--BOLD'})
    #condition_element = soup.find('span', {'data-testid': 'ux-textual-display'})
    price_element = soup.find('div', {'data-testid': 'x-price-primary'})
    #seller_element = soup.find('span', {'class': 'ux-textspans ux-textspans--PSEUDOLINK ux-textspans--BOLD'})
    reviews_element = soup.find('div', {'class': 'x-review-details__allreviews'})
    
    # Get the text from the elements
    product_title = product_title_element.get_text(strip=True)
    #condition = condition_element.find(class_="ux-textspans").text.strip()
    price = price_element.find(class_="ux-textspans").text.strip()
    #seller = seller_element.get_text(strip=True)
    reviews_link = reviews_element.find('a', {'data-testid': 'ux-action'})['href']
    reviews_count = int(reviews_element.find('a', {'data-testid': 'ux-action'}).text.strip().split()[2])
    if reviews_count % 10 == 0:
        pages_count = int(reviews_count/10)
    else:
        pages_count = int(reviews_count/10)+1
    pages_list = list()
    
    for i in range(pages_count):
        page = reviews_link + "&pgn=" + str(i+1)
        response = requests.get(page)
        
        html_content = response.content

        # Create a BeautifulSoup object
        soup = BeautifulSoup(html_content, 'html.parser')

        # Find all the review elements
        review_elements = soup.find_all(class_="ebay-review-section")
        

        # Loop through the review elements and extract the text
        for review_element in review_elements:
            try:
                review_text = review_element.find(class_="review-item-content").get_text(strip=True, separator='\n')
                
            except:
                review_text = ""
                pass
            try:
                review_title = review_element.find(class_="review-item-title").get_text(strip=True, separator='\n')
            except:
                review_title = ""
                pass
            try:
                review_star = review_element.find(class_="star-rating")['aria-label']
            except:
                review_star = ""
            try:
                review_author = review_element.find(class_="review-item-author").get_text(strip=True, separator='\n')
            except:
                review_author = ""
                pass
            try:
                review_date = review_element.find(class_="review-item-date").get_text(strip=True, separator='\n')
            except:
                review_date = ""
                pass
            try:
                review_verifiedPurchase = review_element.find_all(class_="rvw-val")[0].get_text(strip=True, separator='\n')
            except:
                review_verifiedPurchase = ""
                pass
            try:
                condition = review_element.find_all(class_="rvw-val")[1].get_text(strip=True, separator='\n')
            except:
                condition = ""
                pass
            try:
                seller = review_element.find_all(class_="rvw-val")[2].get_text(strip=True, separator='\n')
            except:
                seller = ""
                pass
            results.append({'Product Title': product_title, 'Condition': condition, 'Price': price, 'Seller': seller, 
                            'Review': review_text, 'Review - Title': review_title, 'Review - Star': review_star,
                            'Review - Author': review_author, 'Review - Date': review_date,  
                            'Review - Verified Purchase': review_verifiedPurchase})

df = pd.DataFrame(results)
df


,Product Title,Condition,Price,Seller,Review,Review - Title,Review - Star,Review - Author,Review - Date,Review - Verified Purchase
0,Apple iPhone 8 - 64GB - Space Grey (Unlocked) ...,pre-owned,£110.00,musicmagpie,"Ideal size of iPhone for every day use, very g...",Ideal size great battery life easy charge on a...,5 stars,gordon7168,"10 Sep, 2019",Yes
1,Apple iPhone 8 - 64GB - Space Grey (Unlocked) ...,pre-owned,£110.00,steeley4232,"As with most apple products, it works well wit...","Works well, just the right size, Qi charging i...",4 stars,trevtherev23,"30 Jan, 2020",Yes
2,Apple iPhone 8 - 64GB - Space Grey (Unlocked) ...,new,£110.00,techdustry,"apple are awesome... easy to use, great batter...",iPhone 8 brand new,5 stars,onyxvikkim,"16 Feb, 2019",Yes
3,Apple iPhone 8 - 64GB - Space Grey (Unlocked) ...,pre-owned,£110.00,superbayer35,I like this product firstly because it’s compa...,Good small factor phone,5 stars,muralichava09,"12 Jul, 2022",Yes
4,Apple iPhone 8 - 64GB - Space Grey (Unlocked) ...,pre-owned,£110.00,innofinity-worl...,iphone in excellent condition as described.Add...,Lightweight and easy set up.,4 stars,jsandis10,"24 Aug, 2022",Yes
...,...,...,...,...,...,...,...,...,...,...
385,Apple iPhone 8 - 64GB - Space Grey (Unlocked) ...,new,£110.00,sirgreavus,Excellent phone,Ka,5 stars,nozmul,"05 Dec, 2019",Yes
386,Apple iPhone 8 - 64GB - Space Grey (Unlocked) ...,pre-owned,£110.00,tech-soul,Perfect,iPhone 8 excellent,5 stars,tanyajasmin2012,"05 Apr, 2019",Yes
387,Apple iPhone 8 - 64GB - Space Grey (Unlocked) ...,pre-owned,£110.00,twinnieh,Excellent,iPhone,5 stars,simonvcoyle2013,"23 Oct, 2018",Yes
388,Apple iPhone 8 - 64GB - Space Grey (Unlocked) ...,new,£110.00,forelecplus88,Good,Good,4 stars,orangetrucker1948,"23 Jun, 2020",Yes
